<a href="https://colab.research.google.com/github/dabingooo/Trax-examples/blob/master/trax_transformer_%E5%B0%86%E5%90%84%E7%A7%8D%E6%97%A5%E6%9C%9F%E6%A0%BC%E5%BC%8F%E8%BD%AC%E4%B8%BA%E6%A0%87%E5%87%86%E6%A0%BC%E5%BC%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 将各种日期格式转为标准格式 by Trax Transformer模型
本代码是作为使用google的trax库的例子. 本来是打算做机器翻译(中文->英文), 但因为资源不足,训练时间长,且很可能没有好的结果.所以将问题简化为将各种日期格式->标准日期的训练.

**转化示例**:
> 各种日期格式&nbsp;&nbsp;&nbsp;->&nbsp;&nbsp;标准日期格式<br/>
'10.11.19'&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;->&nbsp;&nbsp;'2019-11-10'<br/>
'1970/9/10'&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;->&nbsp;&nbsp;'1970-09-10'<br/>
'1990年4月28日星期六'&nbsp;&nbsp;->&nbsp;&nbsp;'1990-04-28'<br/>


In [ ]:
import os
import numpy as np
!pip install -q -U trax
import trax
from trax import layers as tl
!pip install -q faker
!pip install -q tqdm
!pip install -q babel

     |████████████████████████████████| 419kB 5.9MB/s 
     |████████████████████████████████| 1.5MB 12.6MB/s 
     |████████████████████████████████| 163kB 45.2MB/s 
     |████████████████████████████████| 2.6MB 53.7MB/s 
     |████████████████████████████████| 194kB 62.5MB/s 
     |████████████████████████████████| 81kB 12.1MB/s 
     |████████████████████████████████| 358kB 39.3MB/s 
     |████████████████████████████████| 368kB 63.4MB/s 
     |████████████████████████████████| 983kB 63.6MB/s 
     |████████████████████████████████| 5.3MB 63.3MB/s 
     |████████████████████████████████| 655kB 63.9MB/s 
     |████████████████████████████████| 307kB 55.8MB/s 
     |████████████████████████████████| 1.1MB 55.9MB/s 
     |████████████████████████████████| 3.5MB 59.5MB/s 
     |████████████████████████████████| 1.0MB 52.5MB/s 
     |████████████████████████████████| 71kB 11.0MB/s 
     |████████████████████████████████| 51kB 8.8MB/s 
     |████████████████████████████████| 235kB 64.6MB/

下面函数用于生成数据.此代码源自吴恩达Deep-Learning-Specialization-Coursera课程,此处代码拷贝自[Here](https://github.com/AdalbertoCq/Deep-Learning-Specialization-Coursera/blob/master/Sequence%20Models/week3/Neural%20machine%20translation%20with%20attention/nmt_utils.py)
,有一些小改动..

In [ ]:
import numpy as np
from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date

fake = Faker()
Faker.seed(12345)
random.seed(12345)

# Define format of the data we would like to generate
FORMATS = ['short',
           'medium',
           'long',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'd MMM YYY', 
           'd MMMM YYY',
           'dd MMM YYY',
           'd MMM, YYY',
           'd MMMM, YYY',
           'dd, MMM YYY',
           'd MM YY',
           'd MMMM YYY',
           'MMMM d YYY',
           'MMMM d, YYY',
           'dd.MM.YY']

# change this if you want it to work with another language
LOCALES = ['en_US']

def load_date():
    """
        Loads some fake dates 
        :returns: tuple containing human readable string, machine readable string, and date object
    """
    dt = fake.date_object()

    try:
        human_readable = format_date(dt, format=random.choice(FORMATS),  locale='zh_CN') # locale=random.choice(LOCALES))
        human_readable = human_readable.lower()
        human_readable = human_readable.replace(',','')
        machine_readable = dt.isoformat()
        
    except AttributeError as e:
        return None, None, None

    return human_readable, machine_readable, dt

def load_dataset(m):
    """
        Loads a dataset with m examples and vocabularies
        :m: the number of examples to generate
    """
    
    human_vocab = set()
    machine_vocab = set()
    dataset = []
    Tx = 30
    
    for i in tqdm(range(m)):
        h, m, _ = load_date()
        if h is not None:
            dataset.append((h, m))
            human_vocab.update(tuple(h))
            machine_vocab.update(tuple(m))
    
    human = dict(zip(['@', '#'] + sorted(human_vocab), 
                     list(range(len(human_vocab) + 2))))
    inv_machine = dict(zip(['@', '#'] + sorted(machine_vocab), 
                     list(range(len(machine_vocab) + 2))))
    machine = {v:k for k,v in inv_machine.items()}
 
    return dataset, human, machine, inv_machine

生成训练数据集:

In [ ]:
m = 120000    # 数据总数量
m_train = 100000 # 用于训练的数量,其余的用于测试时的验证
max_len = 32
batch_size = 32
dataset, human_vocab, inv_machine_vocab, machine_vocab = load_dataset(m)

print('\n========数据集前10个=========')
print(dataset[:10])
print('========可读日期vocab=========')
print(human_vocab)
print('========机器日期vocab=========')
print(machine_vocab)
print('========机器日期 反vocab=========')
print(inv_machine_vocab)


100%|██████████| 120000/120000 [00:19<00:00, 6139.37it/s]


========数据集前10个=========
[('9 5月 1998', '1998-05-09'), ('10.11.19', '2019-11-10'), ('1970/9/10', '1970-09-10'), ('1990年4月28日星期六', '1990-04-28'), ('1995年1月26日星期四', '1995-01-26'), ('1983年3月7日星期一', '1983-03-07'), ('1988年5月22日星期日', '1988-05-22'), ('08 7月 2008', '2008-07-08'), ('8 9月 1999', '1999-09-08'), ('1981年1月1日星期四', '1981-01-01')]
========可读日期vocab=========
{'@': 0, '#': 1, ' ': 2, '.': 3, '/': 4, '0': 5, '1': 6, '2': 7, '3': 8, '4': 9, '5': 10, '6': 11, '7': 12, '8': 13, '9': 14, '一': 15, '七': 16, '三': 17, '九': 18, '二': 19, '五': 20, '八': 21, '六': 22, '十': 23, '四': 24, '年': 25, '日': 26, '星': 27, '月': 28, '期': 29}
========机器日期vocab=========
{'@': 0, '#': 1, '-': 2, '0': 3, '1': 4, '2': 5, '3': 6, '4': 7, '5': 8, '6': 9, '7': 10, '8': 11, '9': 12}
========机器日期 反vocab=========
{0: '@', 1: '#', 2: '-', 3: '0', 4: '1', 5: '2', 6: '3', 7: '4', 8: '5', 9: '6', 10: '7', 11: '8', 12: '9'}


因为trax中的tokenize()和detokenize()函数,如果词汇表类型(vocab_type)使用'char'方式是不能指定自己词汇表(指定了内部也不用).为了使用自己的词汇表所以自己简单实现tokenize()和detokenize()功能:

In [ ]:
def _tok(data, dic):
  '''
  将字符串转为整数token
  data: String类型
  dic: 词汇表字典
  return: np.array类型
  '''
  s = []
  for c in data:
    s.append(dic[c])
  return np.array(s)

def tok_tuple_list(data, dic, axis=0):
  '''
  将 列表中的每个tuplet中的字符串 tokenize
  data: tuple列表, 如[('abc', 'ABC'), ('def', 'fukk')]
  dic: 词汇表字典
  axis: 指定tuple中第几个元素执行tokenize
  return: tokenize后的tuple列表, 如[(np.array([1, 3]), 'ABC'), (np.array([1, 3]), 'fukk')]
  ''' 
  l = []
  for da in data:
    l.append(_tok(da[axis], dic))
  one, two  = zip(*data)
  if axis==0:
    return list(zip(l, two))   
  return list(zip(one, l))

def tok_string_list(data, dic):
  '''
  将 字符串列表 转为 token列表
  data: 字符串列表, 如['abc', 'def']
  dic: 词汇表字典
  return: token列表, 如[np.array([1, 3]), np.array([1, 3])]
  ''' 
  l = []
  for da in data:
    l.append(_tok(da, dic))  
  return l

def detok_string_list(data, dic):
  '''
  将 token列表 转为 字符串列表
  data: token列表, 如[np.array([1, 3]), np.array([1, 3])]
  dic: 词汇表字典
  return: 字符串列表, 如['abc', 'def']
  ''' 
  l = ['' for i in range(data.shape[0])]
  for i in range(data.shape[0]):
    for j in range(data.shape[1]):
      l[i] += dic[data[i][j]]
  return l

print(f'数据集前2个数据: {dataset[:2]}')
input = tok_tuple_list(dataset, human_vocab, 0)
input = tok_tuple_list(input, machine_vocab, 1)
print(f'数据集前2个数据token: {input[:2]}')

#unit test
print('===============unit test: detok_string_list() and detok_string_list()===========================')
tmp_d = ['1998-05-09', '2019-11-10']
print(f'source: {tmp_d}')
tmp_tok = tok_string_list(tmp_d, machine_vocab)
print(f'source_tok: {tmp_tok}')
tmp_tok = np.concatenate((tmp_tok[0], tmp_tok[1])).reshape(2, len(tmp_tok[0]))
source_new = detok_string_list(tmp_tok, inv_machine_vocab)
print(f'source_new: {source_new}')

数据集前2个数据: [('9 5月 1998', '1998-05-09'), ('10.11.19', '2019-11-10')]
数据集前2个数据token: [(array([14,  2, 10, 28,  2,  6, 14, 14, 13]), array([ 4, 12, 12, 11,  2,  3,  8,  2,  3, 12])), (array([ 6,  5,  3,  6,  6,  3,  6, 14]), array([ 5,  3,  4, 12,  2,  4,  4,  2,  4,  3]))]
===============unit test: detok_string_list() and detok_string_list()===========================
source: ['1998-05-09', '2019-11-10']
source_tok: [array([ 4, 12, 12, 11,  2,  3,  8,  2,  3, 12]), array([ 5,  3,  4, 12,  2,  4,  4,  2,  4,  3])]
source_new: ['1998-05-09', '2019-11-10']


数据预处理:

In [ ]:
input_pip = trax.data.Serial(
  trax.data.FilterByLength(max_length=max_len, length_keys=[0, 1]),
  trax.data.BucketByLength(boundaries=[max_len],
              batch_sizes=[batch_size, 1],
              length_keys=[0, 1],
              strict_pad_on_len=True),  
  trax.data.AddLossWeights(id_to_mask=0),
)

train_batches_stream = input_pip(input[:m_train])
eval_batches_stream = input_pip(input[m_train:])

o = next(train_batches_stream)
print(f'第一批数据:{o}')
print(f'第一批数据tuple中每个数据shape: {[x.shape for x in o]}')

第一批数据:(array([[14,  2, 10, ...,  0,  0,  0],
       [ 6,  5,  3, ...,  0,  0,  0],
       [ 6, 14, 12, ...,  0,  0,  0],
       ...,
       [ 6, 14, 13, ...,  0,  0,  0],
       [ 6, 14, 13, ...,  0,  0,  0],
       [ 6, 14, 13, ...,  0,  0,  0]]), array([[ 4, 12, 12, ...,  0,  0,  0],
       [ 5,  3,  4, ...,  0,  0,  0],
       [ 4, 12, 10, ...,  0,  0,  0],
       ...,
       [ 4, 12, 11, ...,  0,  0,  0],
       [ 4, 12, 11, ...,  0,  0,  0],
       [ 4, 12, 11, ...,  0,  0,  0]]), array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]], dtype=float32))
第一批数据tuple中每个数据shape: [(32, 32), (32, 32), (32, 32)]


创建Transformer训练模型:

In [ ]:
# MODEL
def create_model(mode = 'train'):
  return trax.models.Transformer(
      input_vocab_size=len(human_vocab),
      output_vocab_size=len(machine_vocab),
      d_model=32, d_ff=128,
      n_heads=8, n_encoder_layers=2, n_decoder_layers=2,
      max_len=max_len, mode=mode)

# UNUSED
def create_model_reformer(mode = 'train'):
  return trax.models.Reformer(input_vocab_size=8269,
              output_vocab_size=8185,
              d_model=256,
              d_ff=1024,
              n_encoder_layers=2,
              n_decoder_layers=2,
              n_heads=8,
              dropout=0.1,
              max_len=258,
              ff_activation=tl.Relu,
              ff_dropout=None,
              mode=mode)

训练(使用GPU可以加快训练速度):

In [ ]:
# TRAIN
from trax.supervised import training

# Training task.
train_task = training.TrainTask(
    labeled_data=train_batches_stream,
    loss_layer=tl.CrossEntropyLoss(),
    optimizer=trax.optimizers.Adafactor(),
    #optimizer=trax.optimizers.Adam(learning_rate=0.1, weight_decay_rate=1e-05, b1=0.9, b2=0.98, eps=1e-06, clip_grad_norm=None),
    n_steps_per_checkpoint=300,
)

# Evaluaton task.
eval_task = training.EvalTask(
    labeled_data=eval_batches_stream,
    metrics=[tl.CrossEntropyLoss(), tl.Accuracy()],
    n_eval_batches=20  # For less variance in eval numbers.
)

# Training loop saves checkpoints to output_dir.
output_dir = os.path.expanduser('~/output_dir/')
print(output_dir)
!rm -rf {output_dir}
training_loop = training.Loop(create_model(),
                train_task,
                eval_tasks=[eval_task],
                output_dir=output_dir)


# Run
training_loop.run(1800)

/root/output_dir/

Step      1: Ran 1 train steps in 16.44 secs
Step      1: train CrossEntropyLoss |  3.02210116
Step      1: eval  CrossEntropyLoss |  2.46161259
Step      1: eval          Accuracy |  0.20125000

Step    300: Ran 299 train steps in 18.27 secs
Step    300: train CrossEntropyLoss |  0.90951586
Step    300: eval  CrossEntropyLoss |  0.68087703
Step    300: eval          Accuracy |  0.74750000

Step    600: Ran 300 train steps in 4.32 secs
Step    600: train CrossEntropyLoss |  0.37855753
Step    600: eval  CrossEntropyLoss |  0.12818675
Step    600: eval          Accuracy |  0.95859376

Step    900: Ran 300 train steps in 4.37 secs
Step    900: train CrossEntropyLoss |  0.08506602
Step    900: eval  CrossEntropyLoss |  0.06627027
Step    900: eval          Accuracy |  0.97968751

Step   1200: Ran 300 train steps in 4.35 secs
Step   1200: train CrossEntropyLoss |  0.04304006
Step   1200: eval  CrossEntropyLoss |  0.08948441
Step   1200: eval          Accuracy |  0.974687

测试模型效果:

In [ ]:
# 生成测试数据
test_dataset, _, _, _ = load_dataset(10)

for dat in test_dataset:
  # 模型必须每次重新加载, 因为调用autoregressive_sample()时,会改变model的状态值
  model = create_model('predict') 
  model.init_from_file(output_dir+ '/model.pkl.gz', weights_only=True, 
            input_signature=[trax.shapes.ShapeDtype((1, 1), np.int32), 
                      trax.shapes.ShapeDtype((1, 1), np.int32),
                      trax.shapes.ShapeDtype((1, 1), np.float32)])
  # Tokenize a sentence.
  test_source = [dat[0]]
  test_target = [dat[1]]
  test_tok = tok_string_list(test_source, human_vocab)
  test_tok = test_tok[0].reshape(1, len(test_tok[0]))
  #test_tok = trax.data.inputs.pad_to_max_dims(test_tok, boundary=32, strict_pad_on_len=True)

  res_tok = trax.supervised.decoding.autoregressive_sample(
      model, inputs=test_tok, batch_size=1, temperature=0.5, 
      start_id=human_vocab['@'], eos_id=machine_vocab['#'], max_length=10, accelerate=False)

  res = detok_string_list(res_tok, inv_machine_vocab)
  print('======================================================')
  print(f'输入: {test_source}')
  print(f'token(输入): {test_tok}')
  print(f'token(输出): {res_tok}')
  print(f'输出: {res}')
  print(f'真值: {test_target}')

100%|██████████| 10/10 [00:00<00:00, 2755.06it/s]


输入: ['2002年12月3日星期二']
token(输入): [[ 7  5  5  7 25  6  7 28  8 26 27 29 19]]
token(输出): [[5 3 3 5 2 4 5 2 3 6]]
输出: ['2002-12-03']
真值: ['2002-12-03']
输入: ['1990/3/30']
token(输入): [[ 6 14 14  5  4  8  4  8  5]]
token(输出): [[ 4 12 12  3  2  3  6  2  6  3]]
输出: ['1990-03-30']
真值: ['1990-03-30']
输入: ['3 十一月 1991']
token(输入): [[ 8  2 23 15 28  2  6 14 14  6]]
token(输出): [[ 4 12 12  4  2  4  4  2  3 12]]
输出: ['1991-11-09']
真值: ['1991-11-03']
输入: ['1989年12月21日星期四']
token(输入): [[ 6 14 13 14 25  6  7 28  7  6 26 27 29 24]]
token(输出): [[ 4 12 11 12  2  4  5  2  5  4]]
输出: ['1989-12-21']
真值: ['1989-12-21']
输入: ['5 十一月 2011']
token(输入): [[10  2 23 15 28  2  7  5  6  6]]
token(输出): [[5 3 4 4 2 4 4 2 3 8]]
输出: ['2011-11-05']
真值: ['2011-11-05']
输入: ['2000年7月30日星期日']
token(输入): [[ 7  5  5  5 25 12 28  8  5 26 27 29 26]]
token(输出): [[ 5  3  3  3  2  3 10  2  6  3]]
输出: ['2000-07-30']
真值: ['2000-07-30']
输入: ['八月 5 2016']
token(输入): [[21 28  2 10  2  7  5  6 11]]
token(输出): [[ 5  3  4  9  2  3 11  2  3  8